# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Необходимо построить модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span><ul class="toc-item"><li><span><a href="#Удаление-пропусков-и-столбцов" data-toc-modified-id="Удаление-пропусков-и-столбцов-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Удаление пропусков и столбцов</a></span></li><li><span><a href="#Разделение-данных" data-toc-modified-id="Разделение-данных-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Разделение данных</a></span></li><li><span><a href="#Кодирование-столбца-Gender" data-toc-modified-id="Кодирование-столбца-Gender-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Кодирование столбца <code>Gender</code></a></span></li><li><span><a href="#Кодирование-столбца-Geography" data-toc-modified-id="Кодирование-столбца-Geography-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Кодирование столбца <code>Geography</code></a></span></li><li><span><a href="#Масштабирование-признаков-CreditScore,-Balance-и-EstimatedSalary" data-toc-modified-id="Масштабирование-признаков-CreditScore,-Balance-и-EstimatedSalary-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Масштабирование признаков <code>CreditScore</code>, <code>Balance</code> и <code>EstimatedSalary</code></a></span></li></ul></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span><ul class="toc-item"><li><span><a href="#Исследование-баланса-классов" data-toc-modified-id="Исследование-баланса-классов-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Исследование баланса классов</a></span></li><li><span><a href="#Изучение-моделей-без-учёта-дисбаланса" data-toc-modified-id="Изучение-моделей-без-учёта-дисбаланса-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Изучение моделей без учёта дисбаланса</a></span></li></ul></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span><ul class="toc-item"><li><span><a href="#Увеличение-выборки-(upsampling)" data-toc-modified-id="Увеличение-выборки-(upsampling)-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Увеличение выборки (upsampling)</a></span><ul class="toc-item"><li><span><a href="#Проверка-моделей" data-toc-modified-id="Проверка-моделей-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>Проверка моделей</a></span></li></ul></li><li><span><a href="#Уменьшение-выборки-(downsampling)" data-toc-modified-id="Уменьшение-выборки-(downsampling)-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Уменьшение выборки (downsampling)</a></span><ul class="toc-item"><li><span><a href="#Проверка-моделей" data-toc-modified-id="Проверка-моделей-3.2.1"><span class="toc-item-num">3.2.1&nbsp;&nbsp;</span>Проверка моделей</a></span></li></ul></li><li><span><a href="#Автоматическая-балансировка" data-toc-modified-id="Автоматическая-балансировка-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Автоматическая балансировка</a></span></li></ul></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Общий вывод</a></span></li></ul></div>

## Подготовка данных

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.utils import shuffle
from sklearn.preprocessing import OneHotEncoder

In [2]:
df = pd.read_csv('/datasets/Churn.csv')

In [3]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [5]:
df.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,9091.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,4.997690,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.894723,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,2.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


**Описание данных**

Признаки:
- `RowNumber` — индекс строки в данных
- `CustomerId` — уникальный идентификатор клиента
- `Surname` — фамилия
- `CreditScore` — кредитный рейтинг
- `Geography` — страна проживания
- `Gender` — пол
- `Age` — возраст
- `Tenure` — сколько лет человек является клиентом банка
- `Balance` — баланс на счёте
- `NumOfProducts` — количество продуктов банка, используемых клиентом
- `HasCrCard` — наличие кредитной карты
- `IsActiveMember` — активность клиента
- `EstimatedSalary` — предполагаемая зарплата

Целевой признак:
- `Exited` — факт ухода клиента

In [6]:
df.Tenure.value_counts()

1.0     952
2.0     950
8.0     933
3.0     928
5.0     927
7.0     925
4.0     885
9.0     882
6.0     881
10.0    446
0.0     382
Name: Tenure, dtype: int64

In [7]:
df[df.Tenure.isna()]

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
30,31,15589475,Azikiwe,591,Spain,Female,39,NaN,0.00,3,1,0,140469.38,1
48,49,15766205,Yin,550,Germany,Male,38,NaN,103391.38,1,0,1,90878.13,0
51,52,15768193,Trevisani,585,Germany,Male,36,NaN,146050.97,2,0,0,86424.57,0
53,54,15702298,Parkhill,655,Germany,Male,41,NaN,125561.97,1,0,0,164040.94,1
60,61,15651280,Hunter,742,Germany,Male,35,NaN,136857.00,1,0,0,84509.57,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9944,9945,15703923,Cameron,744,Germany,Male,41,NaN,190409.34,2,1,1,138361.48,0
9956,9957,15707861,Nucci,520,France,Female,46,NaN,85216.61,1,1,0,117369.52,1
9964,9965,15642785,Douglas,479,France,Male,34,NaN,117593.48,2,0,0,113308.29,0
9985,9986,15586914,Nepean,659,France,Male,36,NaN,123841.49,2,1,0,96833.00,0


**Промежуточный вывод:** в некоторых строках отсутсвуют значения того, сколько лет человек являлся клиентом банка, потери составили 9%, можно было бы предположить, что это значит, что человек является клиентом меньше года, но в этом столбце также есть значение 0. Поэтому нельзя точно сказать, что означают эти пропуски, и логичнее всего будет удалить эти данные.

Помимо этого в данных присутствуют категориальные признаки: пол и страна.  Эти данные считаю необходимым закодировать. Так же в столбцах со значением баланса на счету, кредитного рейтинга и предполагаемой зарплатой имеется большой разброс данных, их считаю необходимым масштабировать.

Столбцы с фамилией, уникальным идентификатором клиента и порядковым номером - данные, которые мало влияют на факт ухода клиента из банка, считаю логичным не включать их в список признаков.

### Удаление пропусков и столбцов

In [8]:
df = df.dropna().reset_index(drop=True)

In [9]:
df.drop(['RowNumber', 'CustomerId', 'Surname'], axis= 1, inplace= True)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9091 entries, 0 to 9090
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      9091 non-null   int64  
 1   Geography        9091 non-null   object 
 2   Gender           9091 non-null   object 
 3   Age              9091 non-null   int64  
 4   Tenure           9091 non-null   float64
 5   Balance          9091 non-null   float64
 6   NumOfProducts    9091 non-null   int64  
 7   HasCrCard        9091 non-null   int64  
 8   IsActiveMember   9091 non-null   int64  
 9   EstimatedSalary  9091 non-null   float64
 10  Exited           9091 non-null   int64  
dtypes: float64(3), int64(6), object(2)
memory usage: 781.4+ KB


### Разделение данных

In [11]:
target = df['Exited']
features = df.drop('Exited', axis=1)

In [12]:
features_train, features_vt, target_train, target_vt = train_test_split(
    features, target, test_size=0.4, random_state=12345, stratify=target)

In [13]:
features_test, features_valid, target_test, target_valid = train_test_split(
    features_vt, target_vt, test_size=0.5, random_state=12345, stratify=target_vt)

In [14]:
print('Размер features_train:', features_train.shape,
    'Размер target_train:', target_train.shape,
    'Размер features_valid:', features_valid.shape,
    'Размер target_valid:', target_valid.shape,
    'Размер features_test:', features_test.shape,
    'Размер target_test:', target_test.shape, sep='\n')

Размер features_train:
(5454, 10)
Размер target_train:
(5454,)
Размер features_valid:
(1819, 10)
Размер target_valid:
(1819,)
Размер features_test:
(1818, 10)
Размер target_test:
(1818,)


In [15]:
print('Соотношение классов target_train:', target_train.value_counts(),
    'Соотношение классов target_valid:', target_valid.value_counts(),
    'Соотношение классов target_test:', target_test.value_counts(), sep='\n')

Соотношение классов target_train:
0    4342
1    1112
Name: Exited, dtype: int64
Соотношение классов target_valid:
0    1448
1     371
Name: Exited, dtype: int64
Соотношение классов target_test:
0    1447
1     371
Name: Exited, dtype: int64


In [16]:
features_train = features_train.reset_index(drop=True)
target_train = target_train.reset_index(drop=True)
features_valid = features_valid.reset_index(drop=True)
target_valid = target_valid.reset_index(drop=True)
features_test = features_test.reset_index(drop=True)
target_test = target_test.reset_index(drop=True)

### Кодирование столбца `Gender`

In [20]:
enc_gender = OneHotEncoder(drop = 'first', handle_unknown='error', sparse=False)

In [21]:
enc_train_gender = pd.DataFrame(enc_gender.fit_transform(features_train[['Gender']]))
enc_train_gender.columns = enc_gender.get_feature_names()
enc_train_gender.head()

,x0_Male
0,1.0
1,0.0
2,1.0
3,0.0
4,0.0


In [22]:
features_train = features_train.join(enc_train_gender)
features_train.drop('Gender', axis = 1, inplace = True)
features_train.head()

,CreditScore,Geography,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,x0_Male
0,526,France,32,7.0,125540.05,1,0,0,86786.41,1.0
1,500,Spain,47,8.0,128486.11,1,1,0,179227.12,0.0
2,802,Spain,40,4.0,0.00,2,1,1,81908.09,1.0
3,731,Spain,39,2.0,126816.18,1,1,1,74850.93,0.0
4,612,Spain,26,4.0,0.00,2,1,1,179780.74,0.0


In [23]:
features_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5454 entries, 0 to 5453
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      5454 non-null   int64  
 1   Geography        5454 non-null   object 
 2   Age              5454 non-null   int64  
 3   Tenure           5454 non-null   float64
 4   Balance          5454 non-null   float64
 5   NumOfProducts    5454 non-null   int64  
 6   HasCrCard        5454 non-null   int64  
 7   IsActiveMember   5454 non-null   int64  
 8   EstimatedSalary  5454 non-null   float64
 9   x0_Male          5454 non-null   float64
dtypes: float64(4), int64(5), object(1)
memory usage: 426.2+ KB


In [24]:
enc_valid_gender = pd.DataFrame(enc_gender.transform(features_valid[['Gender']]))
enc_valid_gender.columns = enc_gender.get_feature_names()
enc_valid_gender.head()

,x0_Male
0,1.0
1,1.0
2,1.0
3,1.0
4,0.0


In [25]:
features_valid = features_valid.join(enc_valid_gender)
features_valid.drop('Gender', axis = 1, inplace = True)
features_valid.head()

,CreditScore,Geography,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,x0_Male
0,675,Spain,32,10.0,0.00,2,1,0,191545.65,1.0
1,784,Germany,38,1.0,138515.02,1,1,1,171768.76,1.0
2,644,Spain,49,10.0,0.00,2,1,1,145089.64,1.0
3,717,Spain,36,2.0,102989.83,2,0,1,49185.57,1.0
4,677,France,25,3.0,0.00,2,1,0,179608.96,0.0


In [26]:
features_valid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1819 entries, 0 to 1818
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      1819 non-null   int64  
 1   Geography        1819 non-null   object 
 2   Age              1819 non-null   int64  
 3   Tenure           1819 non-null   float64
 4   Balance          1819 non-null   float64
 5   NumOfProducts    1819 non-null   int64  
 6   HasCrCard        1819 non-null   int64  
 7   IsActiveMember   1819 non-null   int64  
 8   EstimatedSalary  1819 non-null   float64
 9   x0_Male          1819 non-null   float64
dtypes: float64(4), int64(5), object(1)
memory usage: 142.2+ KB


In [27]:
enc_test_gender = pd.DataFrame(enc_gender.transform(features_test[['Gender']]))
enc_test_gender.columns = enc_gender.get_feature_names()
enc_test_gender.head()

,x0_Male
0,0.0
1,1.0
2,1.0
3,0.0
4,1.0


In [28]:
features_test = features_test.join(enc_test_gender)
features_test.drop('Gender', axis = 1, inplace = True)
features_test.head()

,CreditScore,Geography,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,x0_Male
0,610,Spain,37,10.0,140363.95,2,1,1,129563.86,0.0
1,661,Germany,41,5.0,122552.48,2,0,1,120646.40,1.0
2,753,Spain,51,4.0,79811.72,2,0,1,68260.27,1.0
3,569,Spain,30,3.0,139528.23,1,1,1,33230.37,0.0
4,597,Spain,38,6.0,115702.67,2,1,1,25059.05,1.0


In [29]:
features_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1818 entries, 0 to 1817
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      1818 non-null   int64  
 1   Geography        1818 non-null   object 
 2   Age              1818 non-null   int64  
 3   Tenure           1818 non-null   float64
 4   Balance          1818 non-null   float64
 5   NumOfProducts    1818 non-null   int64  
 6   HasCrCard        1818 non-null   int64  
 7   IsActiveMember   1818 non-null   int64  
 8   EstimatedSalary  1818 non-null   float64
 9   x0_Male          1818 non-null   float64
dtypes: float64(4), int64(5), object(1)
memory usage: 142.2+ KB


### Кодирование столбца `Geography`

In [33]:
enc_geography = OneHotEncoder(drop = 'first', handle_unknown='error', sparse=False)

In [34]:
enc_train_geo = pd.DataFrame(enc_geography.fit_transform(features_train[['Geography']]))
enc_train_geo.columns = enc_geography.get_feature_names()
enc_train_geo.head()

,x0_Germany,x0_Spain
0,0.0,0.0
1,0.0,1.0
2,0.0,1.0
3,0.0,1.0
4,0.0,1.0


In [35]:
features_train = features_train.join(enc_train_geo)
features_train.drop('Geography', axis = 1, inplace = True)
features_train.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,x0_Male,x0_Germany,x0_Spain
0,526,32,7.0,125540.05,1,0,0,86786.41,1.0,0.0,0.0
1,500,47,8.0,128486.11,1,1,0,179227.12,0.0,0.0,1.0
2,802,40,4.0,0.00,2,1,1,81908.09,1.0,0.0,1.0
3,731,39,2.0,126816.18,1,1,1,74850.93,0.0,0.0,1.0
4,612,26,4.0,0.00,2,1,1,179780.74,0.0,0.0,1.0


In [36]:
features_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5454 entries, 0 to 5453
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      5454 non-null   int64  
 1   Age              5454 non-null   int64  
 2   Tenure           5454 non-null   float64
 3   Balance          5454 non-null   float64
 4   NumOfProducts    5454 non-null   int64  
 5   HasCrCard        5454 non-null   int64  
 6   IsActiveMember   5454 non-null   int64  
 7   EstimatedSalary  5454 non-null   float64
 8   x0_Male          5454 non-null   float64
 9   x0_Germany       5454 non-null   float64
 10  x0_Spain         5454 non-null   float64
dtypes: float64(6), int64(5)
memory usage: 468.8 KB


In [37]:
enc_valid_geo = pd.DataFrame(enc_geography.transform(features_valid[['Geography']]))
enc_valid_geo.columns = enc_geography.get_feature_names()
enc_valid_geo.head()

,x0_Germany,x0_Spain
0,0.0,1.0
1,1.0,0.0
2,0.0,1.0
3,0.0,1.0
4,0.0,0.0


In [38]:
features_valid = features_valid.join(enc_valid_geo)
features_valid.drop('Geography', axis = 1, inplace = True)
features_valid.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,x0_Male,x0_Germany,x0_Spain
0,675,32,10.0,0.00,2,1,0,191545.65,1.0,0.0,1.0
1,784,38,1.0,138515.02,1,1,1,171768.76,1.0,1.0,0.0
2,644,49,10.0,0.00,2,1,1,145089.64,1.0,0.0,1.0
3,717,36,2.0,102989.83,2,0,1,49185.57,1.0,0.0,1.0
4,677,25,3.0,0.00,2,1,0,179608.96,0.0,0.0,0.0


In [39]:
features_valid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1819 entries, 0 to 1818
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      1819 non-null   int64  
 1   Age              1819 non-null   int64  
 2   Tenure           1819 non-null   float64
 3   Balance          1819 non-null   float64
 4   NumOfProducts    1819 non-null   int64  
 5   HasCrCard        1819 non-null   int64  
 6   IsActiveMember   1819 non-null   int64  
 7   EstimatedSalary  1819 non-null   float64
 8   x0_Male          1819 non-null   float64
 9   x0_Germany       1819 non-null   float64
 10  x0_Spain         1819 non-null   float64
dtypes: float64(6), int64(5)
memory usage: 156.4 KB


In [40]:
enc_test_geo = pd.DataFrame(enc_geography.transform(features_test[['Geography']]))
enc_test_geo.columns = enc_geography.get_feature_names()
enc_test_geo.head()

,x0_Germany,x0_Spain
0,0.0,1.0
1,1.0,0.0
2,0.0,1.0
3,0.0,1.0
4,0.0,1.0


In [41]:
features_test = features_test.join(enc_test_geo)
features_test.drop('Geography', axis = 1, inplace = True)
features_test.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,x0_Male,x0_Germany,x0_Spain
0,610,37,10.0,140363.95,2,1,1,129563.86,0.0,0.0,1.0
1,661,41,5.0,122552.48,2,0,1,120646.40,1.0,1.0,0.0
2,753,51,4.0,79811.72,2,0,1,68260.27,1.0,0.0,1.0
3,569,30,3.0,139528.23,1,1,1,33230.37,0.0,0.0,1.0
4,597,38,6.0,115702.67,2,1,1,25059.05,1.0,0.0,1.0


In [42]:
features_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1818 entries, 0 to 1817
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      1818 non-null   int64  
 1   Age              1818 non-null   int64  
 2   Tenure           1818 non-null   float64
 3   Balance          1818 non-null   float64
 4   NumOfProducts    1818 non-null   int64  
 5   HasCrCard        1818 non-null   int64  
 6   IsActiveMember   1818 non-null   int64  
 7   EstimatedSalary  1818 non-null   float64
 8   x0_Male          1818 non-null   float64
 9   x0_Germany       1818 non-null   float64
 10  x0_Spain         1818 non-null   float64
dtypes: float64(6), int64(5)
memory usage: 156.4 KB


### Масштабирование признаков `CreditScore`, `Balance` и `EstimatedSalary`

In [47]:
numeric = ['CreditScore', 'Balance', 'EstimatedSalary'] 

In [48]:
scaler = StandardScaler()
scaler.fit(features_train[numeric])

StandardScaler()

In [49]:
pd.options.mode.chained_assignment = None
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric]) 
features_test[numeric] = scaler.transform(features_test[numeric]) 

In [50]:
features_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5454 entries, 0 to 5453
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      5454 non-null   float64
 1   Age              5454 non-null   int64  
 2   Tenure           5454 non-null   float64
 3   Balance          5454 non-null   float64
 4   NumOfProducts    5454 non-null   int64  
 5   HasCrCard        5454 non-null   int64  
 6   IsActiveMember   5454 non-null   int64  
 7   EstimatedSalary  5454 non-null   float64
 8   x0_Male          5454 non-null   float64
 9   x0_Germany       5454 non-null   float64
 10  x0_Spain         5454 non-null   float64
dtypes: float64(7), int64(4)
memory usage: 468.8 KB


## Исследование задачи

Прогноз, уйдёт клиент из банка в ближайшее время или нет - задача классификации. В проекте будет рассмотрено 3 вида модели: 
- дерево решений;
- случайный лес;
- логистическая регрессия.

### Исследование баланса классов

In [51]:
print('Общее соотношение классов:', target.value_counts(),
    'Соотношение классов в обучающей выборке:', target_train.value_counts(),
    'Соотношение классов в валидационной выборке:', target_valid.value_counts(),
    'Соотношение классов в тестовой выборке:', target_valid.value_counts(), sep='\n')

Общее соотношение классов:
0    7237
1    1854
Name: Exited, dtype: int64
Соотношение классов в обучающей выборке:
0    4342
1    1112
Name: Exited, dtype: int64
Соотношение классов в валидационной выборке:
0    1448
1     371
Name: Exited, dtype: int64
Соотношение классов в тестовой выборке:
0    1448
1     371
Name: Exited, dtype: int64


**Вывод:** в данных наблюдается явный дисбаланс классов, клиентов, оставшихся в банке, примерно в 4 раза больше. Идеальная матрица ошибок должна выглядеть так: [[1448 0] [0 371]].

### Изучение моделей без учёта дисбаланса

In [52]:
# дерево решений (model DecisionTreeClassifier == model_dtc)
best_model_dtc = None
best_depth_dtc = 0
best_result_f1_dtc = 0

for depth in range(1, 11):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train) 
    predictions_valid = model.predict(features_valid) 
    result_f1 = f1_score(target_valid, predictions_valid)
    
    if result_f1 > best_result_f1_dtc:
        best_model_dtc = model
        best_depth_dtc = depth
        best_result_f1_dtc = result_f1


predictions_valid_dtc = best_model_dtc.predict(features_valid)

probabilities_valid_dtc = best_model_dtc.predict_proba(features_valid)[:, 1]
auc_roc_dtc = roc_auc_score(target_valid, probabilities_valid_dtc)

print('Значения наилучшей модели:')
print('Глубина дерева =', best_depth_dtc)
print('F1-метрика =', best_result_f1_dtc)
print('Матрица ошибок:')
print(confusion_matrix(target_valid, predictions_valid_dtc))
print('AUC-ROC-метрика =', auc_roc_dtc)

Значения наилучшей модели:
Глубина дерева = 7
F1-метрика = 0.5611015490533563
Матрица ошибок:
[[1401   47]
 [ 208  163]]
AUC-ROC-метрика = 0.8155248991079805


In [53]:
# случайный лес (model RandomForestClassifier == model_rfc)
best_model_rfc = None
best_est_rfc = 0
best_depth_rfc = 0
best_result_f1_rfc = 0

for est in range(5, 51, 5):
    for depth in range (1, 11):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model.fit(features_train, target_train) 
        predictions_valid = model.predict(features_valid) 
        result_f1 = f1_score(target_valid, predictions_valid)
    
        if result_f1 > best_result_f1_rfc:
            best_model_rfc = model 
            best_est_rfc = est
            best_depth_rfc = depth
            best_result_f1_rfc = result_f1


predictions_valid_rfc = best_model_rfc.predict(features_valid)

probabilities_valid_rfc = best_model_rfc.predict_proba(features_valid)[:, 1]
auc_roc_rfc = roc_auc_score(target_valid, probabilities_valid_rfc)   

print('Значения наилучшей модели:')
print('Количество деревьев =', best_est_rfc)
print('Глубина дерева =', best_depth_rfc)
print('F1-метрика =', best_result_f1_rfc)
print('Матрица ошибок:')
print(confusion_matrix(target_valid, predictions_valid_rfc))
print('AUC-ROC-метрика =', auc_roc_rfc)

Значения наилучшей модели:
Количество деревьев = 25
Глубина дерева = 8
F1-метрика = 0.5828970331588132
Матрица ошибок:
[[1413   35]
 [ 204  167]]
AUC-ROC-метрика = 0.8499575583386695


In [54]:
# логистическая регрессия (model LogisticRegression == model_lr)
model_lr = LogisticRegression(random_state=12345, solver='liblinear', max_iter=1000)
model_lr.fit(features_train, target_train) 
predictions_valid_lr = model_lr.predict(features_valid)
result_f1_lr = f1_score(target_valid, predictions_valid_lr)

probabilities_valid_lr = model_lr.predict_proba(features_valid)[:, 1]
auc_roc_lr = roc_auc_score(target_valid, probabilities_valid_lr)

print('Значения модели:')
print('F1-метрика =', result_f1_lr)
print('Матрица ошибок:')
print(confusion_matrix(target_valid, predictions_valid_lr))
print('AUC-ROC-метрика =', auc_roc_lr)

Значения модели:
F1-метрика = 0.3508771929824562
Матрица ошибок:
[[1396   52]
 [ 281   90]]
AUC-ROC-метрика = 0.7743015740644221


___

In [55]:
metrics_ser = pd.Series(['F1', 'AUC-ROC'])
dtc_ser = pd.Series([best_result_f1_dtc, auc_roc_dtc])
rft_ser = pd.Series([best_result_f1_rfc, auc_roc_rfc])
lr_ser = pd.Series([result_f1_lr, auc_roc_lr])

metrics_df = metrics_ser.to_frame(name='metrics')
dtc_df = dtc_ser.to_frame(name='DTC')
rft_df = rft_ser.to_frame(name='RFT')
lr_df = lr_ser.to_frame(name='LR')

metrics_imbalance = pd.concat([metrics_df, dtc_df, rft_df, lr_df], axis= 1)

In [56]:
metrics_imbalance

,metrics,DTC,RFT,LR
0,F1,0.561102,0.582897,0.350877
1,AUC-ROC,0.815525,0.849958,0.774302


**Вывод:** самые высокие значения метрик F1 и AUC-ROC получились у модели случайного леса: 0.58 и 0.85 соответственно; самые низкие - у модели логистической регрессии: 0.35 и 0.77. При этом ни одной модели не удалось достичь требуемого качества (f1 > 0.59).

## Борьба с дисбалансом

Устранить дисбаланс можно разными методами: 
- увеличением выборки; 
- уменьшением выборки; 
- автоматической балансировкой.

Чтобы наглядно увидеть, как сбалансированная выборка меняет качество модели, возмем модели с лучшими параметрами из шага 2 и обучим их на новых данных, либо применим автобалансировку. 

Для этого создадим функцию, которая будет принимать название модели, обучающие данные и необходимость в автоматической балансировке, а отдавать метрики качества: F1-меру и AUC-ROC.

In [57]:
# Функция вызова 
def get_model_metrics(name_model, features_train_new, target_train_new, wght):
    if wght == True:
        if name_model == 'dtc':
            model = DecisionTreeClassifier(
                random_state=12345, max_depth=5, class_weight='balanced')
        elif name_model == 'rfc':
            model = RandomForestClassifier(
                random_state=12345, n_estimators=20, max_depth=10, class_weight='balanced')
        elif name_model == 'lr':
            model = LogisticRegression(
                random_state=12345, solver='liblinear', max_iter=1000, class_weight='balanced')
        else:
            return print('Функция не работает с такой моделью')
    elif wght == False: 
        if name_model == 'dtc':
            model = DecisionTreeClassifier(random_state=12345, max_depth=5)
        elif name_model == 'rfc':
            model = RandomForestClassifier(random_state=12345, n_estimators=20, max_depth=10)
        elif name_model == 'lr':
            model = LogisticRegression(random_state=12345, solver='liblinear', max_iter=1000)
        else:
            return print('Функция не работает с такой моделью')
    else:
        return print('Не понятно: взвешивать или нет?')
        
    model.fit(features_train_new, target_train_new)
    predictions_valid = model.predict(features_valid) 
    
    result_f1 = f1_score(target_valid, predictions_valid)

    probabilities_valid = model.predict_proba(features_valid)[:, 1]
    auc_roc = roc_auc_score(target_valid, probabilities_valid)
    
    print('Модель:', name_model)
    print('F1-метрика =', result_f1)
    print('Матрица ошибок:')
    print(confusion_matrix(target_valid, predictions_valid))
    print('AUC-ROC-метрика =', auc_roc)
    
    return result_f1, auc_roc

### Увеличение выборки (upsampling)

Преобразование проходит в несколько этапов:
- разделить обучающую выборку на объекты по классам;
- определить тот класс, который содержит меньше объектов;
- скопировать несколько раз объекты меньшего класса; 
- с учетом полученных данных создать новую обучающую выборку;
- перемешать данные.

При этом если использовать "копирование и размножение" выборки с классом 1, а после этого использовать кросс-валидацию, то может получиться чрезмерно оптимистичный результат.

Так же следует отметить, что увеличение выборки следует проводить на обучающей выборке.

In [58]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

In [59]:
features_upsampled, target_upsampled = upsample(features_train, target_train, 4)

In [60]:
print('Размер features_upsampled', features_upsampled.shape,
    'Размер target_upsampled', target_upsampled.shape,
    'Соотношение классов:', target_upsampled.value_counts(), sep='\n')

Размер features_upsampled
(8790, 11)
Размер target_upsampled
(8790,)
Соотношение классов:
1    4448
0    4342
Name: Exited, dtype: int64


#### Проверка моделей

In [61]:
dtc_ups_f1, dtc_ups_auc = get_model_metrics('dtc', features_upsampled, target_upsampled, False)

Модель: dtc
F1-метрика = 0.5532786885245902
Матрица ошибок:
[[1113  335]
 [ 101  270]]
AUC-ROC-метрика = 0.8257332355437744


In [62]:
rfc_ups_f1, rfc_ups_auc = get_model_metrics('rfc', features_upsampled, target_upsampled, False)

Модель: rfc
F1-метрика = 0.5875
Матрица ошибок:
[[1254  194]
 [ 136  235]]
AUC-ROC-метрика = 0.8460205358073595


In [63]:
lr_ups_f1, lr_ups_auc = get_model_metrics('lr', features_upsampled, target_upsampled, False)

Модель: lr
F1-метрика = 0.5052231718898386
Матрица ошибок:
[[1032  416]
 [ 105  266]]
AUC-ROC-метрика = 0.7781417998242767


___

In [64]:
dtc_ups_ser = pd.Series([dtc_ups_f1, dtc_ups_auc])
rft_ups_ser = pd.Series([rfc_ups_f1, rfc_ups_auc])
lr_ups_ser = pd.Series([lr_ups_f1, lr_ups_auc])

dtc_ups_df = dtc_ups_ser.to_frame(name='DTC_ups')
rft_ups_df = rft_ups_ser.to_frame(name='RFT_ups')
lr_ups_df = lr_ups_ser.to_frame(name='LR_ups')

metrics_upsample = pd.concat([metrics_df, dtc_ups_df, rft_ups_df, lr_ups_df], axis= 1)

In [65]:
metrics_upsample

,metrics,DTC_ups,RFT_ups,LR_ups
0,F1,0.553279,0.587500,0.505223
1,AUC-ROC,0.825733,0.846021,0.778142


In [66]:
# для сравнения
metrics_imbalance

,metrics,DTC,RFT,LR
0,F1,0.561102,0.582897,0.350877
1,AUC-ROC,0.815525,0.849958,0.774302


**Вывод:** значение метрики F1 моделей дерева решений и случайного леса немного выросло, а модели логистической регрессии - увеличилось практически вдвое, при этом самое большое значение по-прежнему у случайного леса. Что касается метрик AUC-ROC, они почти не изменились.

### Уменьшение выборки (downsampling)

In [67]:
def downsample(features, target, fraction): 
    features_zeros = features[target == 0] 
    features_ones = features[target == 1] 
    target_zeros = target[target == 0] 
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])

    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)

    return features_downsampled, target_downsampled

In [68]:
features_downsampled, target_downsampled = downsample(features_train, target_train, 0.25)

In [69]:
print('Размер features_downsampled', features_downsampled.shape,
    'Размер target_downsampled', target_downsampled.shape,
    'Соотношение классов:', target_downsampled.value_counts(), sep='\n')

Размер features_downsampled
(2198, 11)
Размер target_downsampled
(2198,)
Соотношение классов:
1    1112
0    1086
Name: Exited, dtype: int64


#### Проверка моделей

In [70]:
dtc_downs_f1, dtc_downs_auc = get_model_metrics('dtc', features_downsampled, target_downsampled, False)

Модель: dtc
F1-метрика = 0.5463709677419355
Матрица ошибок:
[[1098  350]
 [ 100  271]]
AUC-ROC-метрика = 0.8197001906151806


In [71]:
rfc_downs_f1, rfc_downs_auc = get_model_metrics('rfc', features_downsampled, target_downsampled, False)

Модель: rfc
F1-метрика = 0.5735449735449736
Матрица ошибок:
[[1145  303]
 [ 100  271]]
AUC-ROC-метрика = 0.8396663862042262


In [72]:
lr_downs_f1, lr_downs_auc = get_model_metrics('lr', features_downsampled, target_downsampled, False)

Модель: lr
F1-метрика = 0.5046904315196998
Матрица ошибок:
[[1022  426]
 [ 102  269]]
AUC-ROC-метрика = 0.7790539232476063


___

In [73]:
dtc_downs_ser = pd.Series([dtc_downs_f1, dtc_downs_auc])
rft_downs_ser = pd.Series([rfc_downs_f1, rfc_downs_auc])
lr_downs_ser = pd.Series([lr_downs_f1, lr_downs_auc])

dtc_downs_df = dtc_downs_ser.to_frame(name='DTC_downs')
rft_downs_df = rft_downs_ser.to_frame(name='RFT_downs')
lr_downs_df = lr_downs_ser.to_frame(name='LR_downs')

metrics_downsample = pd.concat([metrics_df, dtc_downs_df, rft_downs_df, lr_downs_df], axis= 1)

In [74]:
metrics_downsample

,metrics,DTC_downs,RFT_downs,LR_downs
0,F1,0.546371,0.573545,0.504690
1,AUC-ROC,0.819700,0.839666,0.779054


In [75]:
# для сравнения
metrics_imbalance

,metrics,DTC,RFT,LR
0,F1,0.561102,0.582897,0.350877
1,AUC-ROC,0.815525,0.849958,0.774302


**Вывод:** значение метрики F1 модели дерева решений выросло незначительно, модели случайного леса - уменьшилось, а логистической регрессии так же, как в случае с увеличением выборки, увеличилось практически вдвое. При этом, самое большое значение по-прежнему получается у случайного леса. Что касается метрик AUC-ROC, они почти не изменились.

### Автоматическая балансировка 

Следует отметить, что автоматическую балансировку (использование метода class_weight) применяют не со сбалансированными данными. Использование сразу двух методов балансирования привело бы к новому дисбалансу, поэтому использовать такие методы нужно взаимоисключающе.

In [76]:
dtc_balanced_f1, dtc_balanced_auc = get_model_metrics('dtc', features_train, target_train, True)

Модель: dtc
F1-метрика = 0.5532786885245902
Матрица ошибок:
[[1113  335]
 [ 101  270]]
AUC-ROC-метрика = 0.8257332355437744


In [77]:
rfc_balanced_f1, rfc_balanced_auc = get_model_metrics('rfc', features_train, target_train, True)

Модель: rfc
F1-метрика = 0.5988700564971752
Матрица ошибок:
[[1323  125]
 [ 159  212]]
AUC-ROC-метрика = 0.8470964691516135


In [78]:
lr_balanced_f1, lr_balanced_auc = get_model_metrics('lr', features_train, target_train, True)

Модель: lr
F1-метрика = 0.5053037608486017
Матрица ошибок:
[[1044  404]
 [ 109  262]]
AUC-ROC-метрика = 0.7780878170094265


___

In [79]:
dtc_balanced_ser = pd.Series([dtc_balanced_f1, dtc_balanced_auc])
rft_balanced_ser = pd.Series([rfc_balanced_f1, rfc_balanced_auc])
lr_balanced_ser = pd.Series([lr_balanced_f1, lr_balanced_auc])

dtc_balanced_df = dtc_balanced_ser.to_frame(name='DTC_balanced')
rft_balanced_df = rft_balanced_ser.to_frame(name='RFT_balanced')
lr_balanced_df = lr_balanced_ser.to_frame(name='LR_balanced')

metrics_balanced = pd.concat([metrics_df, dtc_balanced_df, rft_balanced_df, lr_balanced_df], axis= 1)

In [80]:
metrics_balanced

,metrics,DTC_balanced,RFT_balanced,LR_balanced
0,F1,0.553279,0.598870,0.505304
1,AUC-ROC,0.825733,0.847096,0.778088


In [81]:
# для сравнения
metrics_imbalance

,metrics,DTC,RFT,LR
0,F1,0.561102,0.582897,0.350877
1,AUC-ROC,0.815525,0.849958,0.774302


**Вывод:** значение метрик F1 моделей дерева решений и случайного леса выросло незначительно, логистической регрессии - увеличилось практически вдвое. Самое большое значение по-прежнему получается у случайного леса. Что касается метрик AUC-ROC, они почти не изменились.

___

In [82]:
metrics_upsample

,metrics,DTC_ups,RFT_ups,LR_ups
0,F1,0.553279,0.587500,0.505223
1,AUC-ROC,0.825733,0.846021,0.778142


In [83]:
metrics_downsample

,metrics,DTC_downs,RFT_downs,LR_downs
0,F1,0.546371,0.573545,0.504690
1,AUC-ROC,0.819700,0.839666,0.779054


In [84]:
metrics_balanced

,metrics,DTC_balanced,RFT_balanced,LR_balanced
0,F1,0.553279,0.598870,0.505304
1,AUC-ROC,0.825733,0.847096,0.778088


**Общий вывод:** самое большое значение F1-меры получилось у модели случайного леса при автоматической балансировке. Эту модель будем использовать для финального тестирования. Метрики AUC-ROC во время экспериментов практически не изменялись, скорее всего она малочувствительна к дисбалансу в выборках.

## Тестирование модели

In [85]:
# Подготовка модели для финального тестирования
best_model_test = None
best_est_test = 0
best_depth_test = 0
best_result_f1_test = 0

for est in range(5, 81, 5):
    for depth in range (1, 16):
        model = RandomForestClassifier(
            random_state=1024, n_estimators=est, max_depth=depth, class_weight='balanced')
        model.fit(features_train, target_train) 
        predictions_valid = model.predict(features_valid) 
        result_f1 = f1_score(target_valid, predictions_valid)
    
        if result_f1 > best_result_f1_test:
            best_model_test = model 
            best_est_test = est
            best_depth_test = depth
            best_result_f1_test = result_f1   

print('Значения наилучшей модели:')
print('Количество деревьев =', best_est_test)
print('Глубина дерева =', best_depth_test)
print('F1-метрика =', best_result_f1_test)

Значения наилучшей модели:
Количество деревьев = 65
Глубина дерева = 10
F1-метрика = 0.6220362622036263


In [86]:
predictions_test = best_model_test.predict(features_test)
result_f1_test = f1_score(target_test, predictions_test)

print('F1-метрика на тестовой выборке =', result_f1_test)

F1-метрика на тестовой выборке = 0.627187079407806


## Общий вывод

**1. Обзор и подготовка данных.**

В каждой строке таблицы данных - информация о клиенте «Бета-Банка». Часть из них содержит общие данные о человеке: уникальный идентификатор клиента, фамилию, страну проживания, пол, возраст, кредитный рейтинг, сколько лет человек является клиентом банка и предполагаемую зарплату. Другие - это показатели, которые характеризуют его финансовую составляющую в этом банке: баланс на счёте, количество продуктов банка, используемых клиентом, наличие кредитной карты, активность клиента и факт ухода клиента из банка.

В данных имелись пропуски значений того, сколько лет человек являлся клиентом банка, ввиду отстутствия логической замены, пропуски были удалены, потери составили 9%.

Помимо этого в данных присутствовали категориальные признаки: пол и страна. Их было необходимо преобразовать в количественные, для этого использовались техники прямого кодирования, или отображения (One-Hot Encoding) - для пола и порядкового кодирования (Ordinal Encoding) - для страны.

Так же в столбцах со значением баланса на счету, кредитного рейтинга и предполагаемой зарплатой имелся большой разброс данных, их было необходимо масштабировать. Для этого использован метод стандартизации данных.

Столбцы с фамилией, уникальным идентификатором клиента и порядковым номером - являются данными, которые мало влияют на факт ухода клиента из банка, они были удалены из списка признаков.

В конце все данные были разделены на 3 выборки: обучающую, валидационную и тестовую в соотношении 60:20:20 соответственно. Было выявлено, что в данных есть дисбаланс классов: клиентов, которые не ушли из банка примерно в 4 раза больше.

**2. Исследование качества разных моделей.**

Прогноз, уйдёт клиент из банка в ближайшее время или нет - задача классификации. В проекте рассмотрено 3 вида моделей:
- дерево решений;
- случайный лес;
- логистическая регрессия.

Для сравнения качества модели использовалась F1-метрика. Так же параллельно вычислялись значения AUC-ROC метрики. 

Первое исследование моделей прошло без учета дисбаланса в данных, самое высокое значение F1-метрики получила модель случайного леса с количеством деревьев 20 и глубиной 10.

Для борьбы с дисбалансом опробованы 3 метода:
- увеличения выборки;
- уменьшения выборки;
- атоматической балансировки.

Во всех 3 случаях самые высокие значения F1-меры получила модель случайного леса, самые низкие - модель логистической регрессии. Метрика AUC-ROC во время экспериментов практически не изменялась, скорее всего она малочувствительна к дисбалансу в выборках.

**3. Финальное тестирование.**

Для финального тестирования была выбрана модель, которая получила самое высокое значение F1 - модель случайного леса с автоматической балансировкой (F1 = 0.585).

Проверкой на тестовой выборке было достигнуто значения F1-меры 0.63. Удалось достичь нужного качества модели (более 0.59).